# Data Cleaning : Dirty Cafe Sales Dataset

### This notebook demonstrates how to clean a messy cafe sales dataset using Python and pandas. We'll handle missing values,incorrect data types,duplicates,outliers, and inconsistent categorical values

#### 1. Import Libraries

In [1]:
import pandas as pd
import numpy as np

#### 2.Load the Dataset

In [7]:
df = pd.read_csv(r"C:\Users\Aadit\OneDrive\Desktop\Python Data Analysis\data_cleaning proj\dirty_cafe_sales.csv")
df.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2.0,4.0,Credit Card,Takeaway,2023-09-08
1,TXN_4977031,Cake,4,3.0,12.0,Cash,In-store,2023-05-16
2,TXN_4271903,Cookie,4,1.0,ERROR,Credit Card,In-store,2023-07-19
3,TXN_7034554,Salad,2,5.0,10.0,UNKNOWN,UNKNOWN,2023-04-27
4,TXN_3160411,Coffee,2,2.0,4.0,Digital Wallet,In-store,2023-06-11


#### 3.Inspect the Data

##### Check the data types,summary statistics,missing values and duplicates

In [8]:
# check the data types
print(df.dtypes)

Transaction ID      object
Item                object
Quantity            object
Price Per Unit      object
Total Spent         object
Payment Method      object
Location            object
Transaction Date    object
dtype: object


In [12]:
# Summary statistics (for both numeric and categorical)
print(df.describe(include='all'))

       Transaction ID   Item Quantity Price Per Unit Total Spent  \
count           10000   9667     9862           9821        9827   
unique          10000     10        7              8          19   
top       TXN_8903958  Juice        5            3.0         6.0   
freq                1   1171     2013           2429         979   

        Payment Method  Location Transaction Date  
count             7421      6735             9841  
unique               5         4              367  
top     Digital Wallet  Takeaway          UNKNOWN  
freq              2291      3022              159  


In [14]:
# Check for duplicates
print(df.duplicated().sum())

0


In [18]:
#check for missing values
print(df.isnull().sum())

Transaction ID         0
Item                 333
Quantity             138
Price Per Unit       179
Total Spent          173
Payment Method      2579
Location            3265
Transaction Date     159
dtype: int64


#### 4.Handle Missing Values

##### Decide whether to fill,drop, or to impute missing values

In [19]:
# Lets see which columns are most affected from the absence of values
print((df.isnull().sum()/len(df))*100)

Transaction ID       0.00
Item                 3.33
Quantity             1.38
Price Per Unit       1.79
Total Spent          1.73
Payment Method      25.79
Location            32.65
Transaction Date     1.59
dtype: float64


In [21]:
# lets find the numeric columns
numeric_cols = df.select_dtypes(include=['int64','float64']).columns
print(numeric_cols)

Index([], dtype='object')


In [23]:
# Convert to Numeric Columns
numeric_cols = ['Quantity','Price Per Unit','Total Spent']

# Remove non numeric characters and convert to float
for col in numeric_cols:
    df[col] = df[col].replace('[\$,]','',regex=True) # Remove $ or commas
    df[col] = pd.to_numeric(df[col],errors='coerce') # Convert to float,invalid parsing becomes NaN
    

In [24]:
# Check for missing values now
df[numeric_cols].isnull().sum()

Quantity          479
Price Per Unit    533
Total Spent       502
dtype: int64

In [25]:
# Fill the missing values with median
for col in numeric_cols:
    df[col] = df[col].fillna(df[col].median())

In [26]:
# Lets verify now
print(df[numeric_cols].isnull().sum())

Quantity          0
Price Per Unit    0
Total Spent       0
dtype: int64


In [27]:
df[numeric_cols].head()

,Quantity,Price Per Unit,Total Spent
0,2.0,2.0,4.0
1,4.0,3.0,12.0
2,4.0,1.0,8.0
3,2.0,5.0,10.0
4,2.0,2.0,4.0


In [28]:
# Lets find missing values in Categorical Columns
categorical_cols = df.select_dtypes(include='object').columns
print(df[categorical_cols].isnull().sum())

Transaction ID         0
Item                 333
Payment Method      2579
Location            3265
Transaction Date     159
dtype: int64


In [29]:
# Lets clean Location Column

In [32]:
# In Location Column we have 'ERROR','UNKNOWN'. Lets standardize to 'UNKNOWN'
df['Location']=df['Location'].replace(['ERROR','UNKNOWN'],'UNKNOWN')

In [33]:
# Lets Check value Distribution
print(df['Location'].value_counts())

Takeaway    3022
In-store    3017
UNKNOWN      696
Name: Location, dtype: int64


In [34]:
# Since Takeaway and In-Store are almost equal so its safe to keep them as UNKNOWN

In [62]:
# Lets fill with UNKNOWN
df['Location'] = df['Location'].fillna('UNKNOWN')

In [36]:
# Lets clean Payment method


In [37]:
# Lets replace invalid entries with unknown
df['Payment Method'] = df['Payment Method'].replace([' ','UNKNOWN','ERROR'],'UNKNOWN')

In [38]:
print(df['Payment Method'].value_counts())

Digital Wallet    2291
Credit Card       2273
Cash              2258
UNKNOWN            599
Name: Payment Method, dtype: int64


In [39]:
print(df['Payment Method'].isnull().sum())

2579


In [40]:
print((df['Payment Method']=='').sum())

0


In [41]:
# Lets fill with the most common payment
df['Payment Method'] = df['Payment Method'].fillna(df['Payment Method'].mode()[0])

In [42]:
print(df['Payment Method'].isnull().sum())

0


In [43]:
# Lets clean item column

In [44]:
# Check values
print(df['Item'].value_counts())

Juice       1171
Coffee      1165
Salad       1148
Cake        1139
Sandwich    1131
Smoothie    1096
Cookie      1092
Tea         1089
UNKNOWN      344
ERROR        292
Name: Item, dtype: int64


In [47]:
print(df['Item'].isnull().sum())
print((df['Item']== '').sum())

333
0


In [49]:
# Replace invalid entries with UNKNOWN
df['Item'] = df['Item'].replace(['UNKNOWN','ERROR'],'UNKNOWN')

In [51]:
# Since items is randome lets keep it unknown
df['Item'] = df['Item'].fillna('UNKNOWN')

In [52]:
print(df['Item'].isnull().sum())

0


In [53]:
# Lets clean Transaction Date

In [77]:
# Convert to datetime
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'],errors='coerce')

In [56]:
# Check for missing values
print(df['Transaction Date'].isnull().sum())

460


In [83]:
# Lets fill it with UNKNOWN
df['Transaction Date'] = df['Transaction Date'].fillna('UNKNOWN')

In [84]:
print(df['Transaction Date'].isnull().sum())

0


In [85]:
df.isnull().sum()

Transaction ID      0
Item                0
Quantity            0
Price Per Unit      0
Total Spent         0
Payment Method      0
Location            0
Transaction Date    0
dtype: int64

#### 5.Remove Duplicates

In [86]:
# Find if duplicates exist
print(df.duplicated().sum())

0


#### 6.Handle Outliers
##### How to detect them(IQR method):
###### 1.Find the 25th percentile (Q1) and the 75th percentile (Q3)
###### 2.Calculate IQR = Q3-Q1
###### 3. Anything less than Q1-1.5*IQR or  greater than Q3+1.5*IQR is an outlier

In [87]:
# Lets find outliers in Quantity
Q1 = df['Quantity'].quantile(0.25)
Q3 = df['Quantity'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1-1.5*IQR
upper_bound = Q3+1.5*IQR
outliers = df[(df['Quantity']<lower_bound) | (df['Quantity']>upper_bound)]
print(outliers)

Empty DataFrame
Columns: [Transaction ID, Item, Quantity, Price Per Unit, Total Spent, Payment Method, Location, Transaction Date]
Index: []


In [88]:
# Lets find outliers in Price Per Unit
Q1 = df['Price Per Unit'].quantile(0.25)
Q3 = df['Price Per Unit'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1-1.5*IQR
upper_bound = Q3+1.5*IQR
outliers = df[(df['Price Per Unit']<lower_bound) | (df['Price Per Unit']>upper_bound)]
print(outliers)

Empty DataFrame
Columns: [Transaction ID, Item, Quantity, Price Per Unit, Total Spent, Payment Method, Location, Transaction Date]
Index: []


In [89]:
# Lets find outliers in Total Spent
Q1 = df['Total Spent'].quantile(0.25)
Q3 = df['Total Spent'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1-1.5*IQR
upper_bound = Q3+1.5*IQR
outliers = df[(df['Total Spent']<lower_bound) | (df['Total Spent']>upper_bound)]
print(outliers)

Empty DataFrame
Columns: [Transaction ID, Item, Quantity, Price Per Unit, Total Spent, Payment Method, Location, Transaction Date]
Index: []


In [90]:
df['Total Spent'] = df['Total Spent'].clip(lower=lower_bound, upper=upper_bound)


#### 7.Reset Index

In [91]:
df.reset_index(drop=True,inplace=True)

#### 8.Final Verification

In [92]:
# Check datatypes
print(df.dtypes)

Transaction ID       object
Item                 object
Quantity            float64
Price Per Unit      float64
Total Spent         float64
Payment Method       object
Location             object
Transaction Date     object
dtype: object


In [93]:
# Check for missing values
print(df.isnull().sum())

Transaction ID      0
Item                0
Quantity            0
Price Per Unit      0
Total Spent         0
Payment Method      0
Location            0
Transaction Date    0
dtype: int64


In [94]:
# Qucik summary of numeric data
print(df.describe())

           Quantity  Price Per Unit   Total Spent
count  10000.000000     10000.00000  10000.000000
mean       3.027100         2.95265      8.852050
std        1.384614         1.24396      5.791074
min        1.000000         1.00000      1.000000
25%        2.000000         2.00000      4.000000
50%        3.000000         3.00000      8.000000
75%        4.000000         4.00000     12.000000
max        5.000000         5.00000     24.000000


In [95]:
# Check categorical disributions
print(df['Item'].value_counts())
print(df['Payment Method'].value_counts())
print(df['Location'].value_counts())


Juice       1171
Coffee      1165
Salad       1148
Cake        1139
Sandwich    1131
Smoothie    1096
Cookie      1092
Tea         1089
UNKNOWN      969
Name: Item, dtype: int64
Digital Wallet    4870
Credit Card       2273
Cash              2258
UNKNOWN            599
Name: Payment Method, dtype: int64
UNKNOWN     3961
Takeaway    3022
In-store    3017
Name: Location, dtype: int64


#### 9.Save Cleaned Data

In [ ]:
df.to_csv(r"C:\Users\Aadit\OneDrive\Desktop\Python Data Analysis\data_cleaning proj\Cafe_Sales_Cleaned.csv",index=False)